In [1]:
from Crypto.Cipher import AES
import pandas as pd

# Arquivo criptografado
f = open("sample.txt.CRYPT", "rb")
sample = f.read(296)
f.close()


In [3]:
# Campos do cabeçalhos
size = int.from_bytes(sample[279:271:-1]) # 272-279
key_size = int.from_bytes(sample[11:7:-1]) # 8-11
# key 12-267
# 280 - file 
print(sample[11:7:-1])
print(key_size)
print(size)

b'\x00\x00\x01\x00'
256
524288000


### Árvore VAD
Páginas com conteúdo da pilha e *heap* são páginas de memória privada. Filtrando os nós da árvore VAD por páginas com memória privada, e ordenando pelo endereço virtual em ordem decrescente, obtemos uma lista com as páginas que provavelmente contém conteúdo da pilha de um processo.

In [4]:
# Saída do plugin vadinfo
df = pd.read_csv("vadinfo.txt", sep="\t")

# Obtem lista com nome dos arquivos das páginas de memória
# privada de leitura e escrita ordenadas por endereço virtual
files = df[(df["PrivateMemory"] == 1) & (df["Protection"] == "PAGE_READWRITE")]\
    .sort_values("Start VPN")["File output"].to_list()

" ".join(files)


'pid.5928.vad.0x130000-0x16ffff.dmp pid.5928.vad.0x190000-0x191fff.dmp pid.5928.vad.0x1b0000-0x1b3fff.dmp pid.5928.vad.0x200000-0x3fffff.dmp pid.5928.vad.0x2530000-0x253ffff.dmp pid.5928.vad.0x2540000-0x264afff.dmp pid.5928.vad.0x2650000-0x2757fff.dmp pid.5928.vad.0x400000-0x4fffff.dmp pid.5928.vad.0x510000-0x513fff.dmp pid.5928.vad.0x550000-0x64ffff.dmp pid.5928.vad.0x6b0000-0x6bffff.dmp pid.5928.vad.0x7ff50000-0x7ff58fff.dmp pid.5928.vad.0x7ff60000-0x7ff61fff.dmp pid.5928.vad.0x7ff70000-0x7ff80fff.dmp pid.5928.vad.0x7ff90000-0x7ff91fff.dmp pid.5928.vad.0xa80000-0xa8ffff.dmp'

### Entropia
Uma estimativa do valor de entropia é suficiente para diferenciar regiões de memória com alta entropia. Uma boa estimativa é a quantidade de valores diferentes em uma sequência de bytes.

$$
    \text{entropy}(x) = \sum^n_{i=0}{- p_i \cdot \log_2(x_i)}
$$

In [6]:
def entropy_st(sequence: bytes) -> float:
    """Estima a entropia da sequencia de bytes."""

    # Número de valores únicos
    return len(set(sequence))

### Descriptografia

Para verificar se a chave e vetor de inicialização utilizados estão corretos, deve-se utilizar alguma característica conhecida do arquivo a ser descriptografado. Como conhecemos o conteúdo do arquivo de texto, podemos utilizar o primeiro caractere como confirmação. Em outros casos, é possível se aproveitar de características do tipo de arquivo, como o marcador "JFIF", que aparece na posição 6 em imagens no formato PNG.

A busca torna-se mais eficiente se em vez de tentar descriptografar todo o conteúdo do arquivo, utilizar somente os primeiros 16 bytes, ou seja, o primeiro bloco criptografado pelo algoritmo AES. A decodificação parcial é possível devido as características do algoritmo e do modo CBC.

In [28]:
count = 0

def try_decrypt(data: bytes, key: bytes, iv: bytes):
    """
    Tenta descriptografar os dados, retorna o resultado caso
    consiga decodificar em uma string unicode.
    """
    global count
    count += 1
    cipher = AES.new(key=key, iv=iv, mode=AES.MODE_CBC)
    decrypted = cipher.decrypt(data)
    
    try:
        txt = decrypted.decode()
        if txt[0] == "g":
            return txt
    except UnicodeDecodeError:
        pass

### Limite de entropia
O limite da estimativa de entropia foi determinado por tentativa e erro, definido como 40 bytes.

In [24]:
def scan_decrypt(fname: str, data: bytes, threshold=40, window=48):
    """
    Busca por regiões em que a estimativa de entropia é
    maior que threshold, e tenta descriptografar os dados
    """

    f = open(fname, "rb")
    page = f.read()

    for i in range(0, len(page)):
        seq = page[i:i+window]
        ent = entropy_st(seq)

        if ent >= threshold:
            txt = try_decrypt(data, seq[:32], seq[32:])
            if txt:
                return seq, txt

### Recuperação

In [31]:
data = sample[280:]

for fname in files:
    re = scan_decrypt("vad/" + fname, data)

    if re:
        seq, txt = re
        key = seq[:32]
        iv = seq[32:]

        print(fname, txt)
        break

print(str(count) + " tentativas")

pid.5928.vad.0x400000-0x4fffff.dmp gdlndeygjcyhvcaw
3122955 tentativas


In [32]:
def print_hex(seq):
    for byte in seq:
        txt = "%02x" % (byte)
        print(txt.upper(), end=" ")
    print()

print_hex(key)
print_hex(iv)

D1 7B 33 8B A5 91 B1 A8 3B D0 89 A0 F0 E1 02 02 79 52 9A 99 11 A4 3D 23 EA A0 70 5D 55 60 D6 15 
DE 80 E1 A2 EC E6 A8 73 19 A4 AF 0D 28 9D 3B D6 


### Validação do resultado
Descriptografar o arquivo criptografado para verificar se seu conteúdo é o mesmo que o arquivo original.

In [33]:
f1 = open("sample.txt.CRYPT", "rb")
f2 = open("out.txt", "w")
f1.read(280)

cipher = AES.new(key=key, iv=iv, mode=AES.MODE_CBC)

c = 0
while True:
    block = f1.read(1024)

    if block == b'':
        break

    decrypted = cipher.decrypt(block)

    if len(decrypted) != 1024:
        decrypted = decrypted[:-16] # padding

    f2.write(decrypted.decode())
    
f1.close()
f2.close()   